In [21]:
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
from Functions import import_raw_colour_image, import_raw_depth_image, show_depth_image, show_img

In [18]:
filename = r'C:\Users\Ben\Documents\EngD 1st year\Computer Vision\Group project\Github\depth_estimation\Data Preprocessing\colour\2020-03-22 23_03_16-Window.png'

img = Image.open(filename).convert('RGB')
img.show()